## Scripts to help you upload log files (JSON) to a Postgres database

In [1]:
import json
import glob
import os
from os import listdir
from os.path import isfile, isdir, join
import re
import pprint
import dmp.data.logging as logs
import tqdm
import pandas as pd

In [2]:
f = glob.glob("/Users/jperrsau/data/dmpapps/ctripp/data/exp/*")
for file in tqdm.tqdm(f):
    with open(file) as ff:
        fs = ff.read()
        fs = fs.replace("NaN", "null")
        j = json.loads(fs)
        name = file.split("/")[-1][:-5]
        logs.write(name, j)

100%|██████████| 49813/49813 [05:04<00:00, 163.36it/s]


In [22]:
%%time

sql = "SELECT * FROM public.log where doc->'config'->>'activation' = 'relu' ORDER BY id ASC LIMIT 500"

df = pd.read_sql(sql, logs.engine)

pd.json_normalize(df["doc"])

CPU times: user 891 ms, sys: 15.2 ms, total: 906 ms
Wall time: 15.4 s


,endpoint,iterations,loss,num_classes,num_features,num_inputs,num_observations,num_outputs,num_weights,task,...,history.val_accuracy,history.val_cosine_similarity,history.val_hinge,history.val_kullback_leibler_divergence,history.val_loss,history.val_mean_absolute_error,history.val_mean_squared_error,history.val_mean_squared_logarithmic_error,history.val_root_mean_squared_error,history.val_squared_hinge
0,categorical,17,1.362013e+00,5.0,13,141,105908,5,1100,classification,...,"[0.5031630396842957, 0.5031630396842957, 0.503...","[0.5676053166389465, 0.5680404305458069, 0.567...","[1.0733966827392578, 1.0721873044967651, 1.073...","[1.355014681816101, 1.3529911041259766, 1.3536...","[1.3550204038619995, 1.352996826171875, 1.3536...","[0.27339646220207214, 0.2721872329711914, 0.27...","[0.13558480143547058, 0.1354721039533615, 0.13...","[0.06655289977788925, 0.06653086841106415, 0.0...","[0.3682183623313904, 0.3680654466152191, 0.368...","[1.2089812755584717, 1.2076596021652222, 1.209..."
1,categorical,45,1.105857e+00,7.0,11,11,4898,7,2038,classification,...,"[0.527550995349884, 0.527550995349884, 0.52755...","[0.4083039462566376, 0.44882121682167053, 0.51...","[1.098569393157959, 1.093583583831787, 1.08303...","[1.8703750371932983, 1.7734498977661133, 1.615...","[1.8703837394714355, 1.773458480834961, 1.6150...","[0.241426482796669, 0.23644079267978668, 0.225...","[0.11915967613458633, 0.11467745900154114, 0.1...","[0.05806821584701538, 0.055464159697294235, 0....","[0.34519514441490173, 0.33864060044288635, 0.3...","[1.217729091644287, 1.2082611322402954, 1.1896..."
2,categorical,692,1.851504e-09,4.0,8,26,12958,4,4125,classification,...,"[0.8483796119689941, 0.9185956716537476, 0.925...","[0.7890132665634155, 0.8995746374130249, 0.924...","[1.0058330297470093, 0.8926097750663757, 0.851...","[0.7668946981430054, 0.41889092326164246, 0.29...","[0.7668988108634949, 0.41889476776123047, 0.29...","[0.25583314895629883, 0.14260978996753693, 0.1...","[0.10040660202503204, 0.0494060292840004, 0.03...","[0.048214878886938095, 0.024335244670510292, 0...","[0.3168700039386749, 0.22227466106414795, 0.19...","[1.1062397956848145, 0.9420157670974731, 0.888..."
3,categorical,6,2.800080e-02,10.0,784,784,70000,10,523459,classification,...,"[0.9508571624755859, 0.9651428461074829, 0.969...","[0.9575355648994446, 0.9700688719749451, 0.973...","[0.9160725474357605, 0.9109557271003723, 0.908...","[0.1644745022058487, 0.11607646197080612, 0.10...","[0.16448046267032623, 0.1160813570022583, 0.10...","[0.0160724688321352, 0.0109557481482625, 0.008...","[0.007496758829802275, 0.005328355357050896, 0...","[0.0036616050638258457, 0.002600012579932809, ...","[0.08658381551504135, 0.07299558818340302, 0.0...","[0.9235693216323853, 0.9162840247154236, 0.913..."
4,categorical,55,3.746431e-05,4.0,8,26,12958,4,15853,classification,...,"[0.2391975373029709, 0.772376537322998, 0.9023...","[0.5708445906639099, 0.8250271677970886, 0.924...","[1.0879325866699219, 0.9037855863571167, 0.820...","[1.1585335731506348, 0.5121118426322937, 0.230...","[1.1585378646850586, 0.5121151208877563, 0.230...","[0.33793261647224426, 0.15378567576408386, 0.0...","[0.1685045212507248, 0.07399732619524002, 0.03...","[0.08330949395895004, 0.036275990307331085, 0....","[0.41049301624298096, 0.2720244824886322, 0.18...","[1.256437063217163, 0.9777830243110657, 0.8531..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,continuous,10,1.461649e-02,NaN,8,8,20640,1,15622,regression,...,"[0.00024224806111305952, 0.05620155110955238, ...","[0.9997577667236328, 0.9997577667236328, 0.999...","[0.8141762614250183, 0.8078608512878418, 0.798...","[0.09694480150938034, 0.09130219370126724, 0.0...","[0.03585178032517433, 0.03138026222586632, 0.0...","[0.1320117563009262, 0.12120580673217773, 0.11...","[0.03585178032517433, 0.03138026222586632, 0.0...","[0.014321163296699524, 0.012454315088689327, 0...","[0.18934565782546997, 0.17714475095272064, 0.1...","[0.

In [24]:
%%time

sql = "SELECT doc->'history'->'val_accuracy' FROM public.log where doc->'config'->>'activation' = 'relu' ORDER BY id ASC LIMIT 500"

df = pd.read_sql(sql, logs.engine)


CPU times: user 11.1 ms, sys: 1.31 ms, total: 12.4 ms
Wall time: 37 s


In [25]:
df

,?column?
0,"[0.5031630396842957, 0.5031630396842957, 0.503..."
1,"[0.527550995349884, 0.527550995349884, 0.52755..."
2,"[0.8483796119689941, 0.9185956716537476, 0.925..."
3,"[0.9508571624755859, 0.9651428461074829, 0.969..."
4,"[0.2391975373029709, 0.772376537322998, 0.9023..."
...,...
495,"[0.00024224806111305952, 0.05620155110955238, ..."
496,"[0.7010805010795593, 0.7069271802902222, 0.727..."
497,"[0.6551428437232971, 0.9194285869598389, 0.933..."
498,"[0.7636401057243347, 0.7636401057243347, 0.763..."


In [27]:
%%time

sql = "SELECT name FROM public.log where doc->'config'->>'activation' = 'relu' ORDER BY id ASC LIMIT 500"

df = pd.read_sql(sql, logs.engine)

# this is with an index on doc->config->activation and only returning name.

CPU times: user 2.22 ms, sys: 1.04 ms, total: 3.26 ms
Wall time: 35.8 ms


In [28]:
%%time

sql = "SELECT doc->'history'->'val_accuracy' FROM public.log where doc->'config'->>'activation' = 'relu' ORDER BY id ASC LIMIT 500"

df = pd.read_sql(sql, logs.engine)

# this is with an index on doc->config->activation but returning a timeseries in History.

CPU times: user 12.5 ms, sys: 1.97 ms, total: 14.4 ms
Wall time: 22.8 s


In [ ]:
## Looks like pretty poor performance when trying to return lists from the history. That's pretty sad. Oh well.